In [4]:
!pip install oauth2client


   ---------------------------------------- 0.0/98.2 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/98.2 kB ? eta -:--:--
   ------------------------------------- -- 92.2/98.2 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 98.2/98.2 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/96.9 kB ? eta -:--:--
   ---------------------------------------- 96.9/96.9 kB 5.8 MB/s eta 0:00:00


In [5]:
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import os
import smtplib
from email.message import EmailMessage
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Load Excel Sheet
df = pd.read_excel("greenite2.xlsx")  # Ensure the Excel file has 'Name' and 'Designation' columns
font_path = "Font/Duarose_Serif.otf"

# Define Paths for Different Certificate Templates
TEMPLATES = {
    "Head": "Certificate_template/Core-member-certificate.png",         # Template for Head
    "Volunteer": "Certificate_template/Volunteer-certificate.png",  # Template for Volunteer
    "Member": "Certificate_template/Member-certificate.png"          # Template for Member
}

# Google Sheets API Setup
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(creds)

# Open Google Sheet
sheet = client.open("Certificate_Details").sheet1  # Change name as needed

# Email Credentials (Use App Password if using Gmail)
SENDER_EMAIL = "meet.arora2403@gmail.com"   # Replace with your email
SENDER_PASSWORD = "sqpj ijhe ppcg kxrx"       # Replace with your email app password

# Set up SMTP Server (For Gmail)
server = smtplib.SMTP("smtp.gmail.com", 587)
server.starttls()
server.login(SENDER_EMAIL, SENDER_PASSWORD)

FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'

In [ ]:
# Define Font (Ensure the TTF file exists)
font = ImageFont.truetype(font_path, 40)  # Adjust font size if needed
greenite_font = ImageFont.truetype(font_path, 40)  # Adjust font size if needed


TEXT_POSITIONS = {
    "Head": (1300, 637),        # X, Y coordinates for Head template
    "Volunteer": (1265, 690),   # X, Y coordinates for Volunteer template
    "Member": (1300, 685)       # X, Y coordinates for Member template
}

# Define "Greenite" Text Positions for Each Template
GREENITE_POSITIONS = {
    "Head": (410, 770),        # Adjust as needed for Head template
    "Volunteer": (900, 750),    # Adjust as needed for Volunteer template
    "Member": (1100, 750)        # Adjust as needed for Member template
}

# Ensure 'certificates' folder exists
os.makedirs("greenites", exist_ok=True)

for index, row in df.iterrows():
    name = row["Name"]
    designation = row["Designation"]
    recipient_email = row["Email"]
    year = row["YEAR"]
    contact = row["CONTACT"]
    enrollment = row["Enrollment No"]
    # Select the appropriate template based on Designation
    template_path = TEMPLATES.get(designation, "certificate_template.jpg")  # Default if designation not found

    if not os.path.exists(template_path):
        print(f"Template not found for {designation}, skipping {name}")
        continue  # Skip if template does not exist

    # Load the selected template
    template = Image.open(template_path)

    # Get the correct text position for the designation
    text_x, text_y = TEXT_POSITIONS.get(designation, (500, 300))  # Default position if designation not found
    greenite_x, greenite_y = GREENITE_POSITIONS.get(designation, (200, 100))  # Default position if not found
    # Create a copy of the template for each certificate
    cert = template.copy()

    # Draw Name on Certificate
    draw = ImageDraw.Draw(cert)
    draw.text((text_x, text_y), name, fill="black", font=font)  # Adjust color if needed
    draw.text((greenite_x, greenite_y), "Greenite", fill="black", font=greenite_font)  # Greenite text

    # Save Certificate
    # cert_filename = f"greenites/{name}.png"
    # cert.save(cert_filename)
    pdf_filename = f"certificates/{name}.pdf"
    cert.convert("RGB").save(pdf_filename)  # Convert to RGB for PDF format

    print(f"Certificate saved as PDF for: {name} ({designation})")
    # Create Email Message
    
    msg = EmailMessage()
    msg["Subject"] = "Your Certificate"
    msg["From"] = SENDER_EMAIL
    msg["To"] = recipient_email
    msg.set_content(f"Dear {name},\n\nPlease find attached your certificate.\n\nBest Regards,\nPrakriti MSIT")

    # Attach the Certificate File
    with open(pdf_filename, "rb") as attachment:
        msg.add_attachment(attachment.read(), maintype="image", subtype="pdf", filename=f"{name}.pdf")

    # Send Email
    server.send_message(msg)
    print(f"Email sent to {recipient_email}")

    # Add details to Google Sheet
    sheet.append_row([name, year, enrollment, contact, recipient_email, designation])

print("All certificates generated successfully!")
# Close SMTP Server
server.quit()
print("All emails sent successfully!")

Certificate saved as PDF for: Meet Arora (Head)
Email sent to nipungoel346@gmail.com
Certificate saved as PDF for: Ayush kumar  (Volunteer)
Email sent to nipungoel346@gmail.com
Certificate saved as PDF for: Anuska (Member)
Email sent to nipungoel346@gmail.com
All certificates generated successfully!
All emails sent successfully!
